In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from sentence_transformers import SentenceTransformer

In [ ]:
encoder = SentenceTransformer('all-mpnet-base-v2')

In [ ]:
classes = ["clarify", "email", "link", "schedule", "unknown"]

In [ ]:
# Walk through all files in a directory
import os
lines = []
for class_ in classes:
    # Read contents of file
    with open('data/action_classification/' + class_ + '.txt', 'r') as f:
        lines += f.readlines()

inputs = encoder.encode(lines)
inputs = torch.Tensor(inputs)
print(inputs.shape)

In [ ]:
outputs = []
for class_ in classes:
    output_1he = torch.Tensor([0] * len(classes))
    output_1he[classes.index(class_)] = 1
    for i in range(100):
        outputs.append(output_1he)

outputs = torch.stack(outputs)
print(outputs.shape)

In [ ]:
# Split data
from sklearn.model_selection import train_test_split
inputs_train, inputs_test, outputs_train, outputs_test = train_test_split(inputs, outputs, test_size=0.2, random_state=42)

In [ ]:
class ActionClassifier(torch.nn.Module):
    def __init__(self):
        super(ActionClassifier, self).__init__()
        self.fc1 = nn.Linear(768, 384)
        self.dropout1 = nn.Dropout(0.2)
        self.fc2 = nn.Linear(384, 384)
        self.dropout2 = nn.Dropout(0.2)
        self.fc3 = nn.Linear(384, 384)
        self.dropout3 = nn.Dropout(0.2)
        self.fc4 = nn.Linear(384, 384)
        self.dropout4 = nn.Dropout(0.2)
        self.fc5 = nn.Linear(384, len(classes))
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = F.gelu(self.fc1(x))
        x = self.dropout1(x)
        x = F.gelu(self.fc2(x))
        x = self.dropout2(x)
        x = F.gelu(self.fc3(x))
        x = self.dropout3(x)
        x = F.gelu(self.fc4(x))
        x = self.dropout4(x)
        x = F.gelu(self.fc5(x))
        x = self.softmax(x)
        return x

model = ActionClassifier()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

In [ ]:
best_test_loss = float('inf')
for epoch in range(5000):
    optimizer.zero_grad()
    preds_train = model(inputs_train)
    train_loss = criterion(preds_train, outputs_train)
    train_loss.backward()
    train_acc = (preds_train.argmax(1) == outputs_train.argmax(1)).float().mean()
    optimizer.step()

    preds_test = model(inputs_test)
    test_loss = criterion(preds_test, outputs_test)
    test_acc = (preds_test.argmax(1) == outputs_test.argmax(1)).float().mean()
    if test_loss < best_test_loss:
        best_test_loss = test_loss
        torch.save(model.state_dict(), 'action_classifier.pt')

    print('Epoch {} train loss: {} train acc: {} test loss: {} best test loss: {} test acc: {}'.format(epoch, train_loss, train_acc, test_loss, best_test_loss, test_acc))

In [ ]:
action_classifier = ActionClassifier()
action_classifier.load_state_dict(torch.load('action_classifier.pt'))
action_classifier.eval()

In [ ]:
def infer(text):
    input_ = torch.Tensor(encoder.encode([text]))
    output = action_classifier(input_)
    return classes[output.argmax(1)]